<a href="https://colab.research.google.com/github/Alepescinaa/ScientificTools/blob/main/Project1/Cp2/PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises: physics-informed neural network

Exercise on the implementation of physics-informed neural network.

Date: 2024

Course: 056936 - SCIENTIFIC COMPUTING TOOLS FOR ADVANCED MATHEMATICAL MODELLING (PAGANI STEFANO) [2023-24].

Example adapted from this [notebook](https://colab.research.google.com/drive/1qBrbgevkSBqqYc8bOPiaoJG1MBrBrluN?usp=share_link).


Let us consider the problem

\begin{aligned}
  & v_f *\sqrt(\nabla u\cdot D\nabla u) =1  \,, \quad (x,y) \in [-1.5,1.5] \times [-1.5,1.5]\,\\
\end{aligned}

where $\nu$ is unknown. We consider the PINN framework for solving the state/parameter estimation.

In [1]:
# import required libraries

import tensorflow as tf
import numpy as np
import scipy.io
from tensorflow import keras
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
!pip -q install pyDOE
from pyDOE import lhs  # for latin hypercube sampling

  Preparing metadata (setup.py) ... done


In [2]:
# set seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
!git clone https://github.com/Alepescinaa/ScientificTools
%cd ScientificTools/Project1/Cp2

Cloning into 'ScientificTools'...
remote: Enumerating objects: 364, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 364 (delta 60), reused 1 (delta 1), pack-reused 156
Receiving objects: 100% (364/364), 99.94 MiB | 11.29 MiB/s, done.
Resolving deltas: 100% (98/98), done.
/content/ScientificTools/Project1/Cp2


In [4]:
# loading of the dataset

CP2data = np.load("CP2data.npz")
CP2data = CP2data['arr_0']

In [5]:
# loading of the estimate

CP2estimate = np.load("CP2estimate.npz")
CP2estimate = CP2estimate['arr_0']

In [6]:
CP2estimate[4]

array([0.1226488 , 3.6316356 , 0.84714825])

# Solution

In [7]:
X, Y = np.meshgrid(np.linspace(-1.5,1.5,1501), np.linspace(-1.5,1.5,1501))
X_flat = tf.convert_to_tensor(np.hstack((X.flatten()[:,None],Y.flatten()[:,None])),dtype=tf.float64)
x0 = 1.5
#activation_time = anysotropic_FMM( x0 , y0 , X, Y, sigma_11,sigma_12, sigma_21, sigma_22 )

In [8]:
# PINN loss function
def loss(xcl,ycl,xmeas,ymeas,umeas,param):
    umeas_pred = PINN(tf.concat([xmeas,ymeas],1))
    r_pred   = r_PINN(xcl,ycl,param)

    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.pow(r_pred,2))

    # bc
    mse_bc= tf.pow( PINN( tf.transpose( tf.stack( [tf.constant([1.5],dtype=tf.float64), param[2] ] ) ) ) ,2)

    return mse_r + mse_meas + mse_bc

# neural network weight gradients
@tf.function
def grad(model,xcl,ycl,xmeas,ymeas,umeas,param):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(xcl,ycl,xmeas,ymeas,umeas,param)
        grads = tape.gradient(loss_value,model.trainable_variables)
        grad_param = tape.gradient(loss_value,param)
    return loss_value, grads, grad_param

In [9]:
# collocation points
Ncl = 5000
Xcl = lhs(2,Ncl)
xcl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,0],dtype=tf.float64),axis=-1)
ycl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,1],dtype=tf.float64),axis=-1)
X_coll = tf.concat([xcl,ycl],1)

# measurement points
ind_disp=4
xmeas = CP2data[ind_disp][0]
ymeas = CP2data[ind_disp][1]
tmeas = CP2data[ind_disp][2]
xmeas = tf.constant(xmeas.reshape(20, 1), dtype=tf.float64)
ymeas = tf.constant(ymeas.reshape(20, 1), dtype=tf.float64)
tmeas = tf.constant(tmeas.reshape(20, 1), dtype=tf.float64)

In [14]:
# training loop

from tensorflow.keras import regularizers

regularization_strength = 0.01

PINN = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='tanh', input_shape=(2,),
                          kernel_initializer="glorot_normal",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
    tf.keras.layers.Dense(32, activation='tanh',
                          kernel_initializer="glorot_normal",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
    tf.keras.layers.Dense(1, activation=None,
                          kernel_initializer="glorot_normal",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64)])

In [15]:
# residual computation based on AD
@tf.function
def r_PINN(x,y,param):
    u = PINN(tf.concat([x,y], 1))
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    theta0 = pi/2 - param[0]
    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    D = ((1/param[1])* tf.linalg.matmul(a,tf.transpose(a)) + tf.linalg.matmul(b,tf.transpose(b)))

    return  tf.linalg.matmul(tf.transpose(u_grad), tf.linalg.matmul(D,u_grad)) - 1/100**2 #check gradient shape

# Adam optimizer
tf_optimizer = tf.keras.optimizers.Adam(learning_rate=0.003,beta_1=0.99)

# parameter variable try ( 1.45816415][ 3.84494007][-0.95064015]])
#param 0 -> fiber anlge (-pi/10,pi/10)
#param 1-> aniso (0,1)
#param 2-> source y0 (-1.5,1,5)
pi = tf.constant(np.pi,dtype=tf.float64)
param = tf.Variable([[0.01], [3], [1.5]], trainable=True,dtype=tf.float64)

for iter in range(50000):

  # compute gradients using AD
  loss_value,grads,grad_param = grad(PINN,xcl,ycl,xmeas,ymeas,tmeas,param)

  # update neural network weights
  tf_optimizer.apply_gradients(zip(grads+[grad_param],PINN.trainable_variables+[param]))

  # display intermediate results
  if ((iter+1) % 200 == 0):
    print('iter =  '+str(iter+1))
    #loss_value_np=loss_value.numpy()
    #print('loss = {:.4f}'.format(loss_value_np ))
    tf.print('loss =' , loss_value)

    print(param.numpy())
    PINN_flat = PINN(X_flat)

    mse = 0.0
    time_pred = tf.reshape(PINN_flat, (1501, 1501))
    time_pred = time_pred.numpy()
    xmeas=xmeas.numpy().squeeze()
    ymeas=ymeas.numpy().squeeze()
    tmeas=tmeas.numpy().squeeze()

    for k in range(20):
            i, j = np.where((X == xmeas[k]) & (Y == ymeas[k]))
            mse+= (time_pred[i, j] - tmeas[k]) ** 2

    print('mse error: %.4e' % (np.sqrt(mse/20)))

    xmeas = tf.constant(xmeas.reshape(20, 1), dtype=tf.float64)
    ymeas = tf.constant(ymeas.reshape(20, 1), dtype=tf.float64)
    tmeas = tf.constant(tmeas.reshape(20, 1), dtype=tf.float64)

iter =  200
loss = [[0.0055370738377029747]]
[[-0.05540283]
 [ 3.40005342]
 [ 1.31387592]]


<ipython-input-15-4ed302246a48>:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('mse error: %.4e' % (np.sqrt(mse/20)))


mse error: 3.3009e-02
iter =  400
loss = [[0.00041962664157325208]]
[[-0.10578618]
 [ 3.52363156]
 [ 1.26460705]]
mse error: 1.8219e-02
iter =  600
loss = [[0.00013580343066028588]]
[[-0.11486959]
 [ 3.54608523]
 [ 1.25921787]]
mse error: 6.8480e-03
iter =  800
loss = [[2.251799970975174e-05]]
[[-0.11639698]
 [ 3.54986305]
 [ 1.25791093]]
mse error: 4.7506e-03
iter =  1000
loss = [[1.8662848439813362e-05]]
[[-0.11663798]
 [ 3.55043613]
 [ 1.25771448]]
mse error: 4.3183e-03
iter =  1200
loss = [[1.4355635469426409e-05]]
[[-0.1166683 ]
 [ 3.55048681]
 [ 1.25768985]]
mse error: 3.7507e-03
iter =  1400
loss = [[1.1249861734246271e-05]]
[[-0.1166637 ]
 [ 3.55045515]
 [ 1.25771053]]
mse error: 3.3438e-03
iter =  1600
loss = [[8.90951790600341e-06]]
[[-0.11665313]
 [ 3.55041356]
 [ 1.2577817 ]]
mse error: 2.9822e-03
iter =  1800
loss = [[7.05869746807947e-06]]
[[-0.11664177]
 [ 3.5503753 ]
 [ 1.25792516]]
mse error: 2.6532e-03
iter =  2000
loss = [[5.6047915949742733e-06]]
[[-0.1166303 ]
 [ 3

KeyboardInterrupt: 

In [ ]:
#Display results

N_h = 150
X_plot, Y_plot = np.meshgrid(np.linspace(-1.5,1.5,1501), np.linspace(-1.5,1.5,1501))

fig = plt.figure(figsize=(16,9),dpi=150)
#fig = plt.figure()
#fig.subplots_adjust(wspace=0.3)
plt.style.use('default')
ax = fig.add_subplot(1,3,1)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, u_true)
plt.scatter(xmeas,ymeas,marker='x',s=3)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
#ax.set_yticklabels(['-1.0','-0.6','-0.2','0.2','0.6','1.0'])
ax.set_title('Exact Solution',fontsize=16)

ax = fig.add_subplot(1,3,2)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, np.reshape(PINN_flat,(N_h,N_h)))
plt.scatter(xmeas,ymeas,marker='x',s=3)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
ax.set_title('PINN Prediction'.format(err),fontsize=16)

ax = fig.add_subplot(1,3,3)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, np.abs( np.reshape(PINN_flat,(N_h,N_h)) -u_true ) )
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
ax.set_title('L2 error = {:.4f}'.format(err),fontsize=16)